In [11]:
import pandas as pd
import numpy as np
import time, datetime
from binance.client import Client

client = Client()
coin = 'BTCUSDT'
tf = '4h' # 1d 1M 1w 15m

In [12]:
def GetData(symbol='BTCUSDT', start='2020-01-01', tf='1h'):
    frame = pd.DataFrame(client.get_historical_klines(symbol, tf, start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame.set_index('Time', inplace=True)
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [13]:
df = GetData(coin, '2022-01-01', tf)

In [15]:
df['SMA_100'] = df.Close.rolling(100).mean()
df.dropna(inplace=True)

In [16]:
df['Buy'] = (df.Close > df.High.shift(1)) & (df.Close > df.High.shift(2)) & (df.Close > df.SMA_100)

In [17]:
df['Shifted_open'] = df.Open.shift(-1)

In [33]:
is_open_position = False
buy_dates, sell_dates = [], []
buy_prices, sell_prices = [], []
last_trades_time = dict()
    
def MainCycle(sym='BTCUSDT'):
    global is_open_position, buy_dates, sell_dates, buy_prices, sell_prices, tf, last_trades_time
    
    def PeriodSeconds(tf):
            period_seconds = 0
            period_dimension = tf[-1]
            period_qty = int(tf[:-1])
            
            if period_dimension == 'm':
                period_seconds = period_qty * 60
            elif period_dimension == 'h':
                period_seconds = period_qty * 60 * 60
            elif period_dimension == 'd':
                period_seconds = period_qty * 60 * 60 * 24
            elif period_dimension == 'w':
                period_seconds = period_qty * 60 * 60 * 24 * 7
            elif period_dimension == 'M':
                period_seconds = int(period_qty * 60 * 60 * 24 * 30.437)
            
            # print(period_seconds)
            return period_seconds

    
    def WasOpenTradeThisBar_1(checking_time):
        checking_time = checking_time.replace(second=0,microsecond=0)
        # print(checking_time)
        for timestamp in buy_dates:
            if timestamp == checking_time:
                return True
        return False
    
    
    def WasOpenTradeThisBar_2(bar_open_time):
        checking_time = 0
        
        for coin in last_trades_time:
            if coin == sym:
                checking_time = last_trades_time[sym]
        
        if checking_time == 0:
            return False
        
        bar_open_time = datetime.datetime.timestamp(bar_open_time.to_pydatetime())
        next_trade_time = bar_open_time + PeriodSeconds(tf)
        if checking_time <= next_trade_time:
            return True
        return False
    
        
    for index, row in df.iterrows():
        if not is_open_position and row.Buy:
            buy_prices.append(row.Shifted_open)
            buy_dates.append(pd.to_datetime(index,unit='s'))
            is_open_position = True
            last_trades_time[sym] = datetime.datetime.timestamp(index.to_pydatetime())

        if is_open_position:
            is_open_position = False
            sell_dates.append(index)
            sell_prices.append(row.Close)

        current_time = datetime.datetime.timestamp(index.to_pydatetime())
        current_time += 30.5
        current_time = datetime.datetime.fromtimestamp(current_time)
            
        if not is_open_position and row.Buy and not WasOpenTradeThisBar_1(current_time):
            buy_prices.append(row.Shifted_open)
            buy_dates.append(pd.to_datetime(current_time))
            is_open_position = True
        
        if not is_open_position and row.Buy and not WasOpenTradeThisBar_2(index):
            buy_prices.append(row.Shifted_open)
            buy_dates.append(pd.to_datetime(current_time))
            is_open_position = True
            last_trades_time[sym] = datetime.datetime.timestamp(index.to_pydatetime())

In [34]:
MainCycle(coin)

In [35]:
buy_dates

[Timestamp('2022-02-04 04:00:00'),
 Timestamp('2022-02-04 12:00:00'),
 Timestamp('2022-02-04 16:00:00'),
 Timestamp('2022-02-04 20:00:00'),
 Timestamp('2022-02-05 12:00:00'),
 Timestamp('2022-02-06 20:00:00'),
 Timestamp('2022-02-07 00:00:00'),
 Timestamp('2022-02-07 12:00:00'),
 Timestamp('2022-02-07 16:00:00'),
 Timestamp('2022-02-08 04:00:00'),
 Timestamp('2022-02-08 20:00:00'),
 Timestamp('2022-02-10 08:00:00'),
 Timestamp('2022-02-11 12:00:00'),
 Timestamp('2022-02-12 16:00:00'),
 Timestamp('2022-02-14 12:00:00'),
 Timestamp('2022-02-15 00:00:00'),
 Timestamp('2022-02-15 08:00:00'),
 Timestamp('2022-02-15 20:00:00'),
 Timestamp('2022-02-28 12:00:00'),
 Timestamp('2022-02-28 16:00:00'),
 Timestamp('2022-02-28 20:00:00'),
 Timestamp('2022-03-09 00:00:00'),
 Timestamp('2022-03-09 04:00:00'),
 Timestamp('2022-03-09 08:00:00'),
 Timestamp('2022-03-09 12:00:00'),
 Timestamp('2022-03-11 08:00:00'),
 Timestamp('2022-03-18 12:00:00'),
 Timestamp('2022-03-18 16:00:00'),
 Timestamp('2022-03-

In [10]:
sell_dates

[Timestamp('2022-01-20 12:00:00'),
 Timestamp('2022-02-04 04:00:00'),
 Timestamp('2022-02-04 12:00:00'),
 Timestamp('2022-02-04 16:00:00'),
 Timestamp('2022-02-04 20:00:00'),
 Timestamp('2022-02-05 12:00:00'),
 Timestamp('2022-02-06 20:00:00'),
 Timestamp('2022-02-07 00:00:00'),
 Timestamp('2022-02-07 12:00:00'),
 Timestamp('2022-02-07 16:00:00'),
 Timestamp('2022-02-08 04:00:00'),
 Timestamp('2022-02-08 20:00:00'),
 Timestamp('2022-02-10 08:00:00'),
 Timestamp('2022-02-11 12:00:00'),
 Timestamp('2022-02-12 16:00:00'),
 Timestamp('2022-02-14 12:00:00'),
 Timestamp('2022-02-15 00:00:00'),
 Timestamp('2022-02-15 08:00:00'),
 Timestamp('2022-02-15 20:00:00'),
 Timestamp('2022-02-28 12:00:00'),
 Timestamp('2022-02-28 16:00:00'),
 Timestamp('2022-02-28 20:00:00'),
 Timestamp('2022-03-09 00:00:00'),
 Timestamp('2022-03-09 04:00:00'),
 Timestamp('2022-03-09 08:00:00'),
 Timestamp('2022-03-09 12:00:00'),
 Timestamp('2022-03-11 08:00:00'),
 Timestamp('2022-03-18 12:00:00'),
 Timestamp('2022-03-